In [2]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
def check_game(array):
    ar = np.append(np.sum(array,1),np.sum(array,0))
    ar = np.append(ar,[sum(np.diag(array)),sum(np.diag(np.fliplr(array)))])
    if 3 in ar:
        return 1
    elif -3 in ar:
        return -1
    elif 0 not in array:
        return 0.5
    else:
        return 0
class ttt():
    def __init__(self):
        self.rand = True
        self.action_space = list(range(9))

    def reset(self):
        self.xo = [-1,1][random.randint(0,1)]
        self.array = np.array([[0,0,0],[0,0,0],[0,0,0]])
        #self.display = display
        if self.xo==1:
            m = random.randint(0,2)
            n = random.randint(0,2)
            self.array[m][n] = 1

        self.steps = 0
        return self.get_state(),self.rand
    def get_state(self):
        return self.array.reshape(9)
    def step(self,ind):

        self.steps+=1
        if self.array[(ind)//3][ind%3]==0:
            self.array[(ind)//3][ind%3] = self.xo*-1
            result = check_game(self.array)

            if result==0:
                m = random.randint(0,2)
                n = random.randint(0,2)

                while(self.array[m][n]!=0):
                    m = random.randint(0,2)
                    n = random.randint(0,2)
                self.array[m][n] = (self.xo)
                result = check_game(self.array)
                if result == self.xo:
                    return self.get_state(),-1,True,False
                elif result == 0.5:
                    return self.get_state(),-0.1,True,False
                else:
                    return self.get_state(),-0.1,False,False
            else:
                if result == self.xo*-1:
                    return self.get_state(),1,True,False
                elif result == 0.5:
                    return self.get_state(),-0.1,True,False
                else:
                    return self.get_state(),-0.1,False,False
        else:
            if self.steps==20:
                return self.get_state(),-1,False,True
            return self.get_state(),-0.5,False,False
env = ttt()
stored = dict()


In [4]:

def minimax(array,xo):
    global stored
    if str(array) in stored.keys():
        return stored[str(array)]
    if check_game(array)!=0:
        return check_game(array),0
    if xo==1:
        best_score = -np.inf
        for i in np.argwhere(array==0):
            y = np.copy(array)
            y[i[0]][i[1]]= 1
            score,act = minimax(y,xo*-1)
            if str(y) not in stored.keys():
                stored[str(y)] = (score,(act))
            if score>best_score:
                best_score = score
                best_action = i
        if str(array) not in stored.keys():
            stored[str(array)] = (best_score,tuple(best_action))
        return best_score,tuple(best_action)
    else:
        best_score = np.inf
        for i in np.argwhere(array==0):
            y = np.copy(array)
            y[i[0]][i[1]]= -1
            score,act = minimax(y,xo*-1)
            if str(y) not in stored.keys():
                stored[str(y)] = (score,act)
            if score<best_score:
                best_score = score
                best_action = i
        if str(array) not in stored.keys():
            stored[str(array)] = (best_score,tuple(best_action))
        return best_score,tuple(best_action)
minimax(np.array([[0,0,0],[0,0,0],[0,0,0]]),1)

(0.5, (0, 0))

In [5]:


# open file for writing, "w" 
f = open("minimax_table.txt","w")

# write json object to file
f.write(str(stored))

# close file
f.close()

In [35]:
with open('dict.txt') as f:

    stored = eval(str(f.read()))

In [6]:
stored

{'[[ 1 -1  1]\n [-1  1 -1]\n [ 1  0  0]]': (1, 0),
 '[[ 1 -1  1]\n [-1  1 -1]\n [-1  1  1]]': (1, 0),
 '[[ 1 -1  1]\n [-1  1 -1]\n [-1  1  0]]': (1, (2, 2)),
 '[[ 1 -1  1]\n [-1  1 -1]\n [ 1  1 -1]]': (1, 0),
 '[[ 1 -1  1]\n [-1  1 -1]\n [ 0  1 -1]]': (1, (2, 0)),
 '[[ 1 -1  1]\n [-1  1 -1]\n [ 0  1  0]]': (1, (2, 0)),
 '[[ 1 -1  1]\n [-1  1 -1]\n [ 0  0  1]]': (1, 0),
 '[[ 1 -1  1]\n [-1  1 -1]\n [ 0  0  0]]': (1, (2, 0)),
 '[[ 1 -1  1]\n [-1  1  1]\n [-1 -1  1]]': (1, 0),
 '[[ 1 -1  1]\n [-1  1  1]\n [-1 -1  0]]': (1, (2, 2)),
 '[[ 1 -1  1]\n [-1  1  1]\n [-1  1 -1]]': (0.5, 0),
 '[[ 1 -1  1]\n [-1  1  1]\n [-1  0 -1]]': (0.5, (2, 1)),
 '[[ 1 -1  1]\n [-1  1  1]\n [-1  0  0]]': (0.5, (2, 2)),
 '[[ 1 -1  1]\n [-1  1  0]\n [-1  1 -1]]': (0.5, (1, 2)),
 '[[ 1 -1  1]\n [-1  1  0]\n [-1  1  0]]': (0.5, (2, 2)),
 '[[ 1 -1  1]\n [-1  1  0]\n [-1  0  1]]': (1, 0),
 '[[ 1 -1  1]\n [-1  1  0]\n [-1  0  0]]': (1, (2, 2)),
 '[[ 1 -1  1]\n [-1  1  1]\n [ 1 -1 -1]]': (1, 0),
 '[[ 1 -1  1]\n [-1  1